# Visualisation des données

In [1]:
import visdom
import numpy as np
import sys
sys.path.append("../TimeSeriesTools")
import utils
from cerberus import Validator
import chart_studio.plotly as py
import plotly.express as px
import plotly.tools as tls
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime

## Connexion à la base MongoDB

In [20]:
db_host= 'localhost'
port = '27018'
db_name='TimeSeriesBench'
coll_name='SmartGridCryolite20190101OneDay'
client = utils.mongodb_connect(db_host, port)
db = client[db_name]
collection = db[coll_name]


Trying to connect to MongoDB server: localhost on port: 27017


## Lecture des données dans la base
Cette fonction est utilisée pour lire tous les documents dans la base de mongodb, mais il reste un problème : mongodb n'est pas une base de données spécialement conçue pour les données de séries temporaires, donc c'est possible d'exister certains données dédoublées avec même timestamps. Soit on définit un règle avant le stockage de mongodb pour supprimer ou modifier les données dédboulées, ou soit on le néglige.

In [31]:
def get_collection_scheme(db_name,coll_name):
    if 'SmartGrid' in coll_name:
        scheme = {"_id":{'required': True},
                  "timestamp":{'type': 'string','required': True,'empty': False},
                  "tagname":{'type': 'string','required': True,'empty': False},
                  "value":{'type': 'string','required': True,'empty': False},
                  "quality":{'type': 'string','required': True,'empty': False}}
        return scheme
    if 'WindPropData' in coll_name:
        scheme =  {"_id":{'required': True},
                   "Heure":{'type': 'string','required': True,'empty': False},
                   "Temps écoulé":{'type': 'string','required': True,'empty': False},
                   "Latitude":{'type': 'string','required': True,'empty': False},
                   "Longitude":{'type': 'string','required': True,'empty': False},
                   "Altitude":{'type': 'string','required': True,'empty': False},
                   "Head_Rel_True North":{'type': 'string','required': True,'empty': False},
                   "Pressure":{'type': 'string','required': True,'empty': False},
                   "Temperature":{'type': 'string','required': True,'empty': False},
                   "Humidity":{'type': 'string','required': True,'empty': False},
                   "MDA Wnd Dir":{'type': 'string','required': True,'empty': False},
                   "MDA Wnd Speed":{'type': 'string','required': True,'empty': False},
                   "MWD Wind Dir":{'type': 'string','required': True,'empty': False},
                   "MWD Wind Speed":{'type': 'string','required': True,'empty': False},
                   "CavityPressure":{'type': 'string','required': True,'empty': False},
                   "CavityTemp":{'type': 'string','required': True,'empty': False},
                   "CH4":{'type': 'string','required': True,'empty': False},
                   "CH4_dry":{'type': 'string','required': True,'empty': False},
                   "C2H6":{'type': 'string','required': True,'empty': False},
                   "C2H6_dry":{'type': 'string','required': True,'empty': False},
                   "13CH4":{'type': 'string','required': True,'empty': False},
                   "H2O":{'type': 'string','required': True,'empty': False},
                   "CO2":{'type': 'string','required': True,'empty': False},
                   "C2C1Ratio":{'type': 'string','required': True,'empty': False},
                   "Delta_iCH4_Raw":{'type': 'string','required': True,'empty': False},
                   "HP_Delta_iCH4_30s":{'type': 'string','required': True,'empty': False},
                   "HP_Delta_iCH4_2min":{'type': 'string','required': True,'empty': False},
                   "HP_Delta_iCH4_5min":{'type': 'string','required': True,'empty': False}
                  }
        return scheme

In [32]:
def find_all_data(col):
    data_list=[]
    for d in collection.find():
        data_list.append(d)
    print(len(data_list), " documents found")
    return data_list  

## Conversion les dates des données à unix timestamp
Les objects de datetime ne peuvent pas directemant utilisés pour tracer l'axis X, donc il faut d'abord convertir les datetime à unix timestamp.

In [33]:
def to_unix_time(dt):
    epoch =  datetime.utcfromtimestamp(7200)
    return (dt - epoch).total_seconds() * 1000

## Nettoyage des données dans la base
Normalement cette étape est faite avant le stockage des données, vu que on ne sait pas encore les règles détaillés pour la validation les données, on insère quand même les données corruptibles dans la base. <br>
Dans cet exemple, la source des données contient une ligne de données qui manque trois champs pour tester la validation des données.<br>
Donc avant visualiser les courbes, on trouve d'abord les données corruptibles et les supprime. 
Les règles de validation temporaires sont : 
* tous les champs sont présentés
* les types des champs sont string sauf "_id" car cerberus ne connaît pas le type `bson.objectid.ObjectId`
* tous les champs ne sont pas string vide

In [34]:
def clean_data(scheme,data):
    v = Validator(scheme)
    for index,item in enumerate(data,start=0):
        res = v.validate(item)
        if (res == False):
            print("corrupt data in line :",index,", error : ",v.errors)
            del data[index]

### Connecter à la base de test et affichhe le nombre total des données trouvées dans la collection désirée

In [35]:
data = find_all_data(collection)

20700  documents found


### Nettoyer les données récupérées
Les informations des données corruptibles seront affichées dans la console :
* position (nbr de ligne)
* nom de champs qui a des problème
* message d'erreur, pourquoi cette partie de données n'est pas validée

In [36]:
scheme = get_collection_scheme(db_name,coll_name)
clean_data(scheme,data)

In [37]:
df = pd.DataFrame(data)
df[0:5]

,_id,timestamp,tagname,value,quality
0,5ed7ca6242c7406317547679,01/01/2019 09:15:12,CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow,1.000000000,100.0
1,5ed7ca6242c740631754767a,01/01/2019 09:15:18,CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow,0.000000000,100.0
2,5ed7ca6242c740631754767b,01/01/2019 09:15:37,CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow,1.000000000,100.0
3,5ed7ca6242c740631754767c,01/01/2019 09:15:43,CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow,0.000000000,100.0
4,5ed7ca6242c740631754767d,01/01/2019 09:15:53,CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow,1.000000000,100.0


In [77]:
df.tagname.unique()

array(['CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow',
       'CRY.CENTRALE_SOLAIRE.CRY_app_prod_pow',
       'CRY.CENTRALE_SOLAIRE.CRY_rea_prod_pow',
       'CRY.TGBT_NORMAL.CRY_act_cons_pow',
       'CRY.TGBT_NORMAL.CRY_app_cons_pow',
       'CRY.TGBT_NORMAL.CRY_rapp_cons_ene',
       'CRY.TGBT_NORMAL.CRY_rea_cons_pow'], dtype=object)

In [72]:
df2 = df.loc[df['tagname'] == 'CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow']
df2[0:10]

,_id,timestamp,tagname,value,quality
0,5ed7ca6242c7406317547679,01/01/2019 09:15:12,CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow,1.000000000,100.0
1,5ed7ca6242c740631754767a,01/01/2019 09:15:18,CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow,0.000000000,100.0
2,5ed7ca6242c740631754767b,01/01/2019 09:15:37,CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow,1.000000000,100.0
3,5ed7ca6242c740631754767c,01/01/2019 09:15:43,CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow,0.000000000,100.0
4,5ed7ca6242c740631754767d,01/01/2019 09:15:53,CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow,1.000000000,100.0
5,5ed7ca6242c740631754767e,01/01/2019 09:15:58,CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow,0.000000000,100.0
6,5ed7ca6242c740631754767f,01/01/2019 09:16:13,CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow,1.000000000,100.0
7,5ed7ca6242c7406317547680,01/01/2019 09:16:23,CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow,0.000000000,100.0
8,5ed7ca6242c7406317547681,01/01/2019 09:16:53,CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow,1.000000000,100.0
9,5ed7ca6242c7406317547682,01/01/2019 09:16:59,CRY.CENTRALE_SOLAIRE.CRY_act_prod_pow,0.000000000,100.0


### Convertir les string d'heure à datetime object 

In [73]:
dates_list = [datetime.strptime(date, '%d/%m/%Y %H:%M:%S') for date in df2['timestamp']]
dates_list[0:5]

[datetime.datetime(2019, 1, 1, 9, 15, 12),
 datetime.datetime(2019, 1, 1, 9, 15, 18),
 datetime.datetime(2019, 1, 1, 9, 15, 37),
 datetime.datetime(2019, 1, 1, 9, 15, 43),
 datetime.datetime(2019, 1, 1, 9, 15, 53)]

### Préparer la figure basée sur la bibliothèque `plotly`
Pour éviter les données dédoublées, on recupère les premières 5000 lignes comme la source de données du plot.<br>
Dans cet plot les courbes partagent les axis-X et axis-Y, donc on peut observer que les tendances des courbes ne sont pas très claire parce que les plages de valeur des colonnes de données sont très variées.<br>

In [ ]:
x_ref = dates_list[0:5000]
x=[str(xx) for xx in x_ref]
fig = go.Figure()
fig.add_trace(go.Scatter(
                x=x,
                y=[float(item.replace(',', '.')) for item in df["CH4"][0:5000]],
                name="CH4",
                line_color='deepskyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=x,
                y=[float(item.replace(',', '.')) for item in df["Humidity"][0:5000]],
                name="Humidity",
                line_color='dimgray',
                opacity=0.8))

# Use date string to set xaxis range
fig.update_layout(xaxis_range=[to_unix_time(x_ref[0]),
                               to_unix_time(x_ref[-1])],
                  title_text="éolienne data series")
fig.show()

In [74]:
%matplotlib inline
nb_pts=10000
x_ref = dates_list[0:nb_pts]
x=[str(xx) for xx in x_ref]
fig = go.Figure()
fig.add_trace(go.Scatter(
                x=x,
                y=[float(item.replace(',', '.')) for item in df["value"][0:nb_pts]],
                name="value",
                line_color='deepskyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=x,
                y=[float(item.replace(',', '.')) for item in df["quality"][0:nb_pts]],
                name="quality",
                line_color='dimgray',
                opacity=0.8))

# Use date string to set xaxis range
fig.update_layout(xaxis_range=[to_unix_time(x_ref[0]),
                               to_unix_time(x_ref[-1])],
                  title_text="smartgrid data series")
fig.show()

Cet plot contient trois subplots, dans la figure, les courbes partagent l'axis-X mais elles possèdent différents axis-Y.<br>
On remarque que les tendances des courbes sont plus évidentes que la figure précédente.

In [40]:
fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02
)

fig.add_trace(go.Scatter(
                x=x,
                y=[float(item.replace(',', '.')) for item in df["CH4"][0:5000]],
                name="CH4",
                line_color='deepskyblue',
                line_width = 2,
                opacity=0.8),
              row=3, col=1)

fig.add_trace(go.Scatter(
                x=x,
                y=[float(item.replace(',', '.')) for item in df["Humidity"][0:5000]],
                name="Humidity",
                line_color='dimgray',
                opacity=0.8),
              row=2, col=1)

fig.add_trace(go.Scatter(
                x=x,
                y=[float(item.replace(',', '.')) for item in df["13CH4"][0:5000]],
                name="13CH4",
                line_color='rgb(49,130,189)',
                line_width = 1.2,
                opacity=0.8),
              row=1, col=1)

fig.update_layout(height=1000, width=1000,
                  title_text="éolienne data series")
fig.show()

KeyError: 'CH4'

In [53]:
x

['2019-01-01 09:15:12',
 '2019-01-01 09:15:18',
 '2019-01-01 09:15:37',
 '2019-01-01 09:15:43',
 '2019-01-01 09:15:53',
 '2019-01-01 09:15:58',
 '2019-01-01 09:16:13',
 '2019-01-01 09:16:23',
 '2019-01-01 09:16:53',
 '2019-01-01 09:16:59',
 '2019-01-01 09:17:03',
 '2019-01-01 09:17:09',
 '2019-01-01 09:17:18',
 '2019-01-01 09:17:24',
 '2019-01-01 09:17:29',
 '2019-01-01 09:17:34',
 '2019-01-01 09:17:39',
 '2019-01-01 09:17:44',
 '2019-01-01 09:17:55',
 '2019-01-01 09:17:59',
 '2019-01-01 09:18:10',
 '2019-01-01 09:18:14',
 '2019-01-01 09:18:29',
 '2019-01-01 09:18:35',
 '2019-01-01 09:20:16',
 '2019-01-01 09:20:21',
 '2019-01-01 09:20:56',
 '2019-01-01 09:21:01',
 '2019-01-01 09:21:31',
 '2019-01-01 09:21:37',
 '2019-01-01 09:22:53',
 '2019-01-01 09:22:57',
 '2019-01-01 09:23:58',
 '2019-01-01 09:24:03',
 '2019-01-01 09:24:18',
 '2019-01-01 09:24:24',
 '2019-01-01 09:24:28',
 '2019-01-01 09:24:33',
 '2019-01-01 09:25:19',
 '2019-01-01 09:25:29',
 '2019-01-01 09:25:39',
 '2019-01-01 09:

In [75]:
fig = make_subplots(
    rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.02
)

fig.add_trace(go.Scatter(
                x=x[0:nb_pts],
                y=[float(item.replace(',', '.')) for item in df["value"][0:nb_pts]],
                name="value",
                line_color='deepskyblue',
                line_width = 2,
                opacity=0.8),
              row=1, col=1)

fig.update_layout(height=1000, width=1000,
                  title_text="smartgrid data series")
fig.show()

### Initialiser visdom et envoyer le plot à visdom via requête Http post
Avant cette étape, il faut lancer le service de visdom dans le terminal.
```bash
> visdom
```
L'address du web UI de visdom sera affichée dans la console.

In [76]:
vis = visdom.Visdom()
vis.plotlyplot(fig, win="mywin3")

Setting up a new session...


'mywin3'

### Ajuster la taille de la fenêtre

In [ ]:
vis.update_window_opts(win = "mywin3", opts=dict(width=1200, height=1500))

Après l'exécution de ce script, ouvrir le web UI de visdom, le plot est présenté dans une fenêtre intéractive.